In [1]:
# Model calling and intial setup
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser 
import warnings
warnings.filterwarnings("ignore") 

load_dotenv()
# Load env
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
AZURE_BASE_URL = os.getenv("AZURE_BASE_URL")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_CHAT_DEPLIOYMENT_NAME = os.getenv("AZURE_CHAT_DEPLIOYMENT_NAME")

parser = StrOutputParser()

llm_gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash" , api_key= GOOGLE_API_KEY)

llm_openai = AzureChatOpenAI(
    model="gpt-4o-mini",                         
    deployment_name=AZURE_CHAT_DEPLIOYMENT_NAME ,  # deployment name in Azure
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_BASE_URL,
    api_version="2024-02-01"
) 
llm_openai.invoke("What are your creater, also what type of LLM are you").content
# llm_gemini.invoke("who is father of india").content

c:\Users\singh\Let's Gooooo\Langgraph\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


'I was created by OpenAI, an organization focused on advancing artificial intelligence in a safe and beneficial manner. I am a large language model (LLM) known as GPT-3 (Generative Pre-trained Transformer 3). My primary function is to understand and generate human-like text based on the input I receive. If you have any questions or need assistance, feel free to ask!'

### Persistance
- it is refer to ability to save and restore the state of a workflow overtime in langgraph

- Not only store final values, but store all the values, with using reducer fucntion

- This add consistecy and reliability

- Add fault Tolrence

### Checkpointers

- persistance is implemented using checkpointers

- At each superstep, we save state in either RAM or in DB

### Threads

- what about diff users using the application, as for each invoke fucntion i'm saving all the 

- for each unique user an unique chat, we give a new thread id


In [3]:
from langgraph.graph import StateGraph , START , END
from typing import TypedDict
from langgraph.checkpoint.memory import InMemorySaver

class JokeState(TypedDict):
    topic : str
    joke : str
    explanation : str
    
def get_joke(state:JokeState)->JokeState:
    promt = PromptTemplate(
        template="You're a Moderm comedian, based on the topic give a joke that is fresh and new, and more imprtanlt funny, Topic : \n{topic}",
        input_variables=['topic']
    )
    
    joke_create_chain = promt | llm_openai | parser
    joke = joke_create_chain.invoke({"topic" : state['joke']})
    state['joke'] = joke
    return state

def get_explnation(state:JokeState)->JokeState:
    promt = PromptTemplate(
        template="You're a Moderm comedian critic, based on the joke given to you provide a explation to to joke, what is the primise hook and punchline in the joke, Joke: \n{joke}",
        input_variables=['joke']
    )
    
    joke_explain_chain = promt | llm_openai | parser
    explnation = joke_explain_chain.invoke({"joke" : state['joke']})
    state['explanation'] = explnation
    return state

# Create Graph
graph = StateGraph(JokeState)

graph.add_node("create_joke" , get_joke)
graph.add_node("explain_joke" , get_explnation)

# create Edges
graph.add_edge(START , "create_joke")
graph.add_edge("create_joke" , "explain_joke")
graph.add_edge('explain_joke' , END)

# Create checkpointers
checkpinter = InMemorySaver()

joke_workflow = graph.compile(checkpointer=checkpinter)